# Read Avro and covert data to Spark DataFrame

1. Read Avro formated capture data from a Event Hubs, 
1. Covert the byte to string and then to Dataframe

In [97]:
import os
import string
import json
import time
import avro.schema

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

from azure.storage.blob import ContainerClient, BlobClient
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

StatementMeta(spark32m3, 3, 102, Finished, Available)

In [98]:
# Measure time 
start_time = time.time()

StatementMeta(spark32m3, 3, 103, Finished, Available)

In [99]:
# Define UDF to convert byte array to string
def deserializeBody(encodedBody: bytearray):    
    return encodedBody.decode("utf-8")

deserializedBody_udf = udf(deserializeBody, StringType())

StatementMeta(spark32m3, 3, 104, Finished, Available)

In [100]:
# Location of the sample Avro files
PATH = "abfss://datalake@datalakehousepristorage.dfs.core.windows.net/0_bronze/pos/eventdriven-dev/pos/2023/03/*/*/*/*/*.avro"

captured_df = spark.read.format("avro").load(PATH)
display(captured_df)

StatementMeta(spark32m3, 3, 105, Finished, Available)

SynapseWidget(Synapse.DataFrame, 58c709aa-10f7-4c63-8b37-a627ef2e3af8)

In [101]:
# Read Avro and deserialize Body to String
decoded_df = captured_df.withColumn("Body_msg", deserializedBody_udf(captured_df["Body"]))
number_of_events = decoded_df.count()
print(f"Found {number_of_events} rows")

StatementMeta(spark32m3, 3, 106, Finished, Available)

Found 472801 rows


In [102]:
# Display the sample data
display(decoded_df)

StatementMeta(spark32m3, 3, 107, Finished, Available)

SynapseWidget(Synapse.DataFrame, b316a12b-884b-47e4-9a5c-00c883f21be7)

In [103]:
# Just select target column, Body_mag
json_string_df = decoded_df.select("Body_msg")
display(json_string_df)

StatementMeta(spark32m3, 3, 108, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5bb4bfaa-48f9-4870-880e-ffbb734a91b2)

In [104]:
# Define the schema of Json
schema = StructType([StructField("Tid", IntegerType(), True),
                     StructField("Type", StringType(), True),
                     StructField("StoreName", StringType(), True),
                     StructField("SKUName", StringType(), True),
                     StructField("ItemCount", IntegerType(), True),
                     StructField("logidx", IntegerType(), True)])

# Select the column and then covert it to Json object
json_df = json_string_df.select(from_json(col("Body_msg"), schema).alias("data"))
df = json_df.select(col("data.Tid"), col("data.Type"), col("data.StoreName"), col("data.SKUName"), col("data.ItemCount"), col("data.logidx"))
display(df)

StatementMeta(spark32m3, 3, 109, Finished, Available)

SynapseWidget(Synapse.DataFrame, 83462fee-daac-410d-b936-c63d80b6800f)

In [105]:
# Time that taken to prep data
elapsed_time = time.time() - start_time
print("== Result ==")
print(f"Elapsed time: {elapsed_time} seconds to process {number_of_events} rows")

StatementMeta(spark32m3, 3, 110, Finished, Available)

== Result ==
Elapsed time: 34.40555453300476 seconds to process 472801 rows


## End of Notebook